## **Robot Model** 

`RobotModel` is a tiny wrapper around the modeling backend (`casadi_kin_dyn`) that allow you to create symbolical CasAdi model from **urdf** file, then calculate a notions such as **forward and inverse dynamics**, com positions, state space representation etc, one may also calculate **body** specific properties and add **contacts**.

The typical workflow as follows:
* Build the urdf of your robot and create `RobotModel` instance
* Add additional bodies and possibly contacts
* Calculate the all necessary functions with `~.update_model()` method 
* Access to casadi functions that are stored within RobotModel and use them either numerically or symbolically in other CasAdi empowered projects.  

There are also banch of modules that facilitates work with given type of robots, i.e. manipulators, quadrupeds and bipeds.


As example let us create the model of z1 manipulator:

We will use [robot_descriptions.py](https://github.com/robot-descriptions/robot_descriptions.py) library to simplify the process of urdf retrieval.

### *Note*:
You have to restart kernel after `robot_descriptions` is installed

In [1]:
%%capture
!pip3 install robot_descriptions

In [2]:
from darli.backend import CasadiBackend
from darli.modeling import Symbolic, Robot
from robot_descriptions import z1_description

model = Symbolic(CasadiBackend(z1_description.URDF_PATH))

One can retrive basic model info as follows:

In [3]:
nq = model.nq # dimensionality of configuration  
nv = model.nv # dimensionality of generilized velocities
nu = model.nu # dimensionality  of control inputs
q_min, q_max = model.q_min, model.q_max # minimal and maximal limits on joint pos
nq, nv, nu

(6, 6, 6)

In [4]:
joint_names = model.joint_names # names of the joints
joint_names

['universe', 'joint1', 'joint2', 'joint3', 'joint4', 'joint5', 'joint6']

In [5]:
model.joint_id(joint_names[nv-1]) # will return id of the joint by name

5

### **Equations of Motion and Dynamics**

The dynamics of articulated mechanics in robotic systems is usually represented as:
$$
\mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{C}(\mathbf{q},\mathbf{v})\mathbf{v} + \mathbf{g}(\mathbf{q})  = 
\mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{c}(\mathbf{q},\mathbf{v}) + \mathbf{g}(\mathbf{q}) = \mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{h}(\mathbf{q},\mathbf{v}) = \mathbf{Q}
$$

where:
* $\mathbf{Q} \in \mathbb{R}^{nv}$ - generalized forces corresponding to generilized coordinates
* $\mathbf{q} \in \mathbb{R}^{nq}$ - vector of generilized coordinates
* $\mathbf{v} \in \mathbb{R}^{nq}$ - vector of generilized velocities (sometimes $\mathbf{v} = \dot{\mathbf{q}}$, but not in case of $\mathbf{q}$ containing quaternions)
* $\mathbf{M} \in \mathbb{R}^{nv \times nv}$ - positive definite symmetric inertia matrix 
* $\mathbf{c} \in \mathbb{R}^{nv}$ - describe centrifugal and Coriolis forces
* $\mathbf{C} \in \mathbb{R}^{nv \times nv}$ - describe 'coefficients' of centrifugal and Coriolis forces
* $\mathbf{g} \in \mathbb{R}^{nv}$ - describes effect of gravity and other position depending forces
* $\mathbf{h} \in \mathbb{R}^{nv} $ - combined effect of $\mathbf{g}$ and $\mathbf{c}$


One can get all of the above quantities in symbotics as follows:

In [6]:
inertia = model.inertia
gravity_vector = model.gravity
bias_force = model.bias_force
# coriolis_matrix = model.coriolis_matrix
coriolis = model.coriolis

Each of the above define the CasAdi functions:

In [7]:
model.inertia

Function(inertia:(q[6])->(inertia[6x6]) SXFunction)

And can be evaluated both numerically and symbolically:

In [8]:
import numpy as np 
import casadi as cs

# Symbolical computation
print('Symbolical:', inertia(cs.SX.sym('q', nq)))
# Numerical computation
print('Numerical:', inertia(np.random.randn(nq)))

Symbolical: @1=7e-07, @2=sin(q_5), @3=-1.6e-07, @4=cos(q_5), @5=((@1*@2)+(@3*@4)), @6=(@5*@2), @7=0.0001468, @8=0.165803, @9=0.00177591, @10=-0.00017355, @11=-0.00143876, @12=((@10*@4)-(@11*@2)), @13=((0.0005386+(@6+@7))+(@8*(@9+sq(@12)))), @14=0.307927, @15=0.07, @16=0.0491595, @17=cos(q_4), @18=sin(q_4), @19=0.425807, @20=(@19*@12), @21=(@15+((@16*@17)-(@18*@20))), @22=(0.0436668-@21), @23=((@16*@18)+(@17*@20)), @24=(0.00364738-@23), @25=((0.00097912+@13)+(@14*(sq(@22)+sq(@24)))), @26=cos(q_3), @27=2, @28=-1.22e-06, @29=-1.08e-06, @30=-5.4e-07, @31=-0.00698719, @32=((@10*@2)+(@11*@4)), @33=(0.00646316-@32), @34=((-5.689e-05+(((@28*@2)+(@29*@4))-(@30*@4)))-(@31*@33)), @35=(@27*@34), @36=((1.3e-07+((@5*@4)+8e-08))+((@8*@12)*@33)), @37=(@36+@36), @38=(0.0037111+(@19*@32)), @39=(-0.00170192-@38), @40=((-4.091e-05+(((@35*@17)-(@37*@18))-((@17*@34)-(@18*@36))))-((@14*@22)*@39)), @41=sin(q_3), @42=(((@1*@4)-(@3*@2))*@4), @43=(((0.00017605+(((3.718e-05-@42)-@6)+@7))+(@8*(sq(@12)+sq(@33))))-@

Note that above are calculated not via Lagrange formalism but by using efficient recursive Newton-Euler algorithm (for `bias_force`), while inertia matrix is evaluated by composite rigid body algorithm (CRBA)

There are some notions apart from dynamical coefficients that may be useful as well, such as com kinematics and jacobians, and energy:

In [9]:
model.backend.jacobian()

SX(@1=0.673326, @2=2.47e-06, @3=sin(q_0), @4=-0.00025198, @5=cos(q_0), @6=1.19132, @7=-0.110126, @8=cos(q_1), @9=(@3*@8), @10=0.00240029, @11=0.00158266, @12=sin(q_1), @13=(@3*@12), @14=0.839409, @15=0.106092, @16=cos(q_2), @17=sin(q_2), @18=((@9*@16)-(@13*@17)), @19=-0.00541815, @20=0.0347638, @21=((@9*@17)+(@13*@16)), @22=-0.35, @23=(@22*@9), @24=0.564046, @25=0.0436668, @26=cos(q_3), @27=sin(q_3), @28=((@18*@26)-(@21*@27)), @29=0.00364738, @30=-0.00170192, @31=((@18*@27)+(@21*@26)), @32=0.218, @33=0.057, @34=(@23+((@32*@18)+(@33*@21))), @35=0.389385, @36=0.0312153, @37=cos(q_4), @38=sin(q_4), @39=((@28*@37)+(@5*@38)), @40=0.00646316, @41=0.07, @42=(@34+(@41*@28)), @43=0.288758, @44=0.0241569, @45=-0.00017355, @46=((@5*@37)-(@28*@38)), @47=cos(q_5), @48=sin(q_5), @49=-0.00143876, @50=0.0492, @51=(@42+(@50*@39)), @52=(@43*(((@44*@39)+((@45*((@46*@47)+(@31*@48)))+(@49*((@31*@47)-(@46*@48)))))+@51)), @53=((@35*(((@36*@39)+(@40*@31))+@42))+@52), @54=((@24*(((@25*@28)+((@29*@5)+(@30*@31))

In [10]:
model.energy

Energy(kinetic=Function(kinetic_energy:(q[6],v[6])->(kinetic_energy) SXFunction), potential=Function(potential_energy:(q[6])->(potential_energy) SXFunction))

In [11]:
com_position = model.com.position
com_velocity = model.com.velocity
com_jacobian = model.com.jacobian
com_jacobian_dt = model.com.jacobian_dt
potential_energy = model.energy.potential
kinetic_energy = model.energy.kinetic
lagrangian = model.lagrangian

# TODO
# momentum

In [12]:
com_jacobian_dt

Function(com_jacobian_dt:(q[6],v[6])->(com_jacobian_dt[3x6]) SXFunction)

#### **Forward and Inverse Dynamics**


In inverse dynamics we looking for the generilized forces:
$$
    \mathbf{Q} = \mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{h}(\mathbf{q},\mathbf{v}) = \text{rnea}(\mathbf{q}, \mathbf{v}, \dot{\mathbf{v}})
$$

In [13]:
model.contact_qforce

Function(contact_qforce:(q[6])->(contact_qforce[6]) SXFunction)

In [14]:
qforce = 0
for body in model.bodies.values():
    if body.contact is not None:
        qforce += body.contact.qforce

qforce

0

In [15]:
model.inverse_dynamics

Function(inverse_dynamics:(q[6],v[6],dv[6])->(tau[6]) SXFunction)

While forward dynamics is basically solving the equations of motion with respect to accelerations $\dot{\mathbf{v}}$. However the solution of the above in general case usually yields the complexity of $O(nv^3)$, for this reason a way to go is to use celebrated Featherstone Articulated Body algorithm (ABA), which effectevely exploit sparsity of $\mathbf{M}$ for the tree structures and produce nearly linear complexity $O(nv)$:


$$
\dot{\mathbf{v}} = \mathbf{M}^{-1}(\mathbf{q})(\mathbf{Q} - \mathbf{h}(\mathbf{q},\mathbf{v})) = \text{aba}(\mathbf{q}, \mathbf{v}, \mathbf{Q})
$$

<!-- Articulated body algorithm 

Feather stone algorithm
 -->
<!-- http://gamma.cs.unc.edu/courses/robotics-f08/LEC/ArticulatedBodyDynamics.pdf -->

In [16]:
model.contact_qforce

Function(contact_qforce:(q[6])->(contact_qforce[6]) SXFunction)

In [17]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[6])->(dv[6]) SXFunction)

#### **Passive Joints and Selector**

In practical situations 

Choosing the passive joints:

In [18]:
import numpy as np 
print(f'Old input dimensions: {model.nu}')
S = np.random.randn(model.nv, model.nv+2)
model.update_selector(S)
print(f'New input dimensions: {model.nu}')

Old input dimensions: 6
New input dimensions: 8


In [19]:
model.qfrc_u

SX([tau_0, tau_1, tau_2, tau_3, tau_4, tau_5, tau_6, tau_7])

In [20]:
model.selector @ model.qfrc_u

SX([((((((((0.22123*tau_0)+(0.185124*tau_1))+(0.318443*tau_2))+(0.423206*tau_3))+(0.583647*tau_4))+(0.0349872*tau_5))+(1.29241*tau_6))+(-1.67687*tau_7)), ((((((((1.12774*tau_0)+(-0.17889*tau_1))+(-1.32539*tau_2))+(0.705066*tau_3))+(-0.466793*tau_4))+(0.011063*tau_5))+(1.35801*tau_6))+(-0.537413*tau_7)), ((((((((0.536645*tau_0)+(-1.1492*tau_1))+(-0.561958*tau_2))+(0.304179*tau_3))+(-0.818189*tau_4))+(-0.933307*tau_5))+(0.415691*tau_6))+(-0.25235*tau_7)), ((((((((-0.369351*tau_0)+(1.72509*tau_1))+(-1.73768*tau_2))+(-0.22239*tau_3))+(0.298596*tau_4))+(-0.930422*tau_5))+(0.425376*tau_6))+(2.03084*tau_7)), ((((((((0.431506*tau_0)+(0.165456*tau_1))+(1.21628*tau_2))+(-0.733916*tau_3))+(-0.864507*tau_4))+(0.922565*tau_5))+(-1.03386*tau_6))+(0.796051*tau_7)), ((((((((-0.760421*tau_0)+(0.135656*tau_1))+(-1.24441*tau_2))+(-0.766876*tau_3))+(-0.0369025*tau_4))+(2.79429*tau_5))+(-0.444094*tau_6))+(0.412847*tau_7))])

In [21]:
model.selector

array([[ 0.2212297 ,  0.18512398,  0.31844261,  0.42320589,  0.58364707,
         0.03498716,  1.29240645, -1.67686978],
       [ 1.12773677, -0.17888975, -1.32539365,  0.70506566, -0.46679283,
         0.01106303,  1.35801075, -0.53741264],
       [ 0.5366449 , -1.14920043, -0.5619577 ,  0.30417947, -0.8181895 ,
        -0.9333072 ,  0.41569126, -0.25235022],
       [-0.36935074,  1.72508824, -1.73767952, -0.22239043,  0.29859636,
        -0.93042227,  0.42537584,  2.03083877],
       [ 0.4315061 ,  0.16545563,  1.21627817, -0.73391628, -0.8645071 ,
         0.92256477, -1.03385527,  0.79605133],
       [-0.76042142,  0.13565625, -1.24440695, -0.76687596, -0.03690248,
         2.79428725, -0.44409442,  0.41284703]])

In [22]:
model.selector @ model.qfrc_u

SX([((((((((0.22123*tau_0)+(0.185124*tau_1))+(0.318443*tau_2))+(0.423206*tau_3))+(0.583647*tau_4))+(0.0349872*tau_5))+(1.29241*tau_6))+(-1.67687*tau_7)), ((((((((1.12774*tau_0)+(-0.17889*tau_1))+(-1.32539*tau_2))+(0.705066*tau_3))+(-0.466793*tau_4))+(0.011063*tau_5))+(1.35801*tau_6))+(-0.537413*tau_7)), ((((((((0.536645*tau_0)+(-1.1492*tau_1))+(-0.561958*tau_2))+(0.304179*tau_3))+(-0.818189*tau_4))+(-0.933307*tau_5))+(0.415691*tau_6))+(-0.25235*tau_7)), ((((((((-0.369351*tau_0)+(1.72509*tau_1))+(-1.73768*tau_2))+(-0.22239*tau_3))+(0.298596*tau_4))+(-0.930422*tau_5))+(0.425376*tau_6))+(2.03084*tau_7)), ((((((((0.431506*tau_0)+(0.165456*tau_1))+(1.21628*tau_2))+(-0.733916*tau_3))+(-0.864507*tau_4))+(0.922565*tau_5))+(-1.03386*tau_6))+(0.796051*tau_7)), ((((((((-0.760421*tau_0)+(0.135656*tau_1))+(-1.24441*tau_2))+(-0.766876*tau_3))+(-0.0369025*tau_4))+(2.79429*tau_5))+(-0.444094*tau_6))+(0.412847*tau_7))])

In [23]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[8])->(dv[6]) SXFunction)

In [24]:
model.update_selector(passive_joints=range(3))
print(f'New input dimensions: {model.nu}')
print(f'New selector:\n {model.selector}')

New input dimensions: 3
New selector:
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [25]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[3])->(dv[6]) SXFunction)

#### **State space Representation and Linearization**

One can easily transform the mechanical system to the state space form by defining the state $\mathbf{x} = [\mathbf{q}, \mathbf{v}]^T$:


$$
\dot{\mathbf{x}}= \mathbf{f}(\mathbf{x}, \mathbf{u}) = 
\begin{bmatrix}
\dot{\mathbf{x}}_1 \\ 
\dot{\mathbf{x}}_2
\end{bmatrix}=
\begin{bmatrix}
\dot{\mathbf{q}} \\ 
\dot{\mathbf{v}}
\end{bmatrix}=
\begin{bmatrix}
\mathbf{W}(\mathbf{q})\mathbf{v} \\ 
\text{aba}(\mathbf{q}, \mathbf{v}, \mathbf{S}\mathbf{u})
\end{bmatrix}=
\begin{bmatrix}
\mathbf{W}(\mathbf{x}_1)\mathbf{x}_2 \\
\text{aba}(\mathbf{x}_1, \mathbf{x}_2, \mathbf{S}\mathbf{u})
\end{bmatrix}
$$



In [26]:
model.state_space

The above equation can be easily linearized to produce following linear approximation:

One can easily find linearization with respect to state:

In [27]:
model.state_space.state_derivative

SX(@1=2.47e-06, @2=0.673326, @3=-0.00025198, @4=cos(q_1), @5=2.1e-06, @6=-5.13e-06, @7=(@4*v_0), @8=sin(q_1), @9=((v_1*@7)+(@8*dv_0)), @10=0.0246611, @11=(@8*v_0), @12=((@4*dv_0)-(v_1*@11)), @13=-0.110126, @14=1.19132, @15=0.00158266, @16=(@14*((@15*@9)+(@13*@12))), @17=0.00240029, @18=9.81, @19=(@18*@8), @20=(@14*(@19+((@17*@12)-(@15*dv_1)))), @21=0.0242946, @22=-0.00062358, @23=(@14*((@13*v_1)+(@17*@11))), @24=(@14*((@17*@7)-(@15*v_1))), @25=((((@21*v_1)-(@22*@11))+(@5*@7))+((@13*@23)-(@15*@24))), @26=0.00102138, @27=(@14*((@15*@11)+(@13*@7))), @28=((((@22*v_1)-(@26*@11))+(@6*@7))-((@17*@23)+(@15*@27))), @29=cos(q_2), @30=1.332e-05, @31=(dv_2+dv_1), @32=0.00208102, @33=sin(q_2), @34=((@29*@7)-(@33*@11)), @35=((v_2*@34)+((@29*@9)+(@33*@12))), @36=0.00886621, @37=((@29*@11)+(@33*@7)), @38=(((@29*@12)-(@33*@9))-(v_2*@37)), @39=0.106092, @40=0.839409, @41=-0.35, @42=(@41*@12), @43=0.0347638, @44=(@40*(@42+((@43*@35)+(@39*@38)))), @45=-0.00541815, @46=(@41*v_1), @47=(@29*@46), @48=(@18*@4

In [28]:
model.state_space.state_jacobian

SX(@1=cos(q_1), @2=-0.110126, @3=1.19132, @4=0.00158266, @5=cos(q_1), @6=sin(q_1), @7=(v_0*@6), @8=((dv_0*@5)-(v_1*@7)), @9=(v_0*@5), @10=((dv_0*@6)+(v_1*@9)), @11=(@3*((@4*@8)-(@2*@10))), @12=0.00240029, @13=9.81, @14=(@13*@5), @15=(@3*(@14-(@12*@10))), @16=-5.13e-06, @17=0.0246611, @18=-0.00062358, @19=sin(q_1), @20=(@19*v_0), @21=2.1e-06, @22=(@1*v_0), @23=(@3*((@2*v_1)+(@12*@20))), @24=(@3*((@12*@22)-(@4*v_1))), @25=((((0.0242946*v_1)-(@18*@20))+(@21*@22))+((@2*@23)-(@4*@24))), @26=(@3*(@12*@9)), @27=(@3*(@12*@7)), @28=(((@2*@26)+(@4*@27))-((@18*@9)+(@21*@7))), @29=0.00102138, @30=(@3*((@4*@9)-(@2*@7))), @31=(((@29*@9)+(@16*@7))+((@12*@26)+(@4*@30))), @32=cos(q_2), @33=0.106092, @34=0.839409, @35=0.0347638, @36=sin(q_2), @37=((@32*@7)+(@36*@9)), @38=(((@32*@8)-(@36*@10))-(v_2*@37)), @39=((@32*@9)-(@36*@7)), @40=(((@32*@10)+(@36*@8))+(v_2*@39)), @41=-0.35, @42=(@41*@10), @43=(@34*(((@35*@38)-(@33*@40))-@42)), @44=-0.00541815, @45=(@13*@6), @46=((@32*@14)-(@36*@45)), @47=(@34*((@44*@

and control:

In [29]:
model.state_space.input_jacobian

SX(all zero sparse: 12-by-3)

These functionality allows for easy implementation of linearization based analysis and control.

#### **Bodies and Contacts**

In [30]:
model.add_body(['link06'])
model.bodies

{'link06': <darli.modeling.body.Body at 0x7f2593f54610>}

In [31]:
model.body('link06')

One may also retrieve a hash map of all bodies:

In [32]:
model.bodies

{'link06': <darli.modeling.body.Body at 0x7f2593f54610>}

In [33]:
model.body("link06").linear_acceleration.local

SX(@1=sin(q_0), @2=cos(q_1), @3=(@1*@2), @4=cos(q_2), @5=sin(q_1), @6=(@1*@5), @7=sin(q_2), @8=((@3*@4)-(@6*@7)), @9=cos(q_3), @10=((@3*@7)+(@6*@4)), @11=sin(q_3), @12=((@8*@9)-(@10*@11)), @13=cos(q_4), @14=cos(q_0), @15=sin(q_4), @16=((@12*@13)+(@14*@15)), @17=-0.35, @18=(@14*@2), @19=(@17*@18), @20=0.218, @21=(@14*@5), @22=((@18*@4)-(@21*@7)), @23=0.057, @24=((@18*@7)+(@21*@4)), @25=(@19+((@20*@22)+(@23*@24))), @26=0.07, @27=((@22*@9)-(@24*@11)), @28=(@25+(@26*@27)), @29=0.0492, @30=((@27*@13)-(@1*@15)), @31=(@28+(@29*@30)), @32=(@17*@3), @33=(@32+((@20*@8)+(@23*@10))), @34=(@33+(@26*@12)), @35=(@34+(@29*@16)), @36=0.1035, @37=(@36-(@17*@5)), @38=((@2*@4)-(@5*@7)), @39=((@5*@4)+(@2*@7)), @40=(@37+((@23*@38)-(@20*@39))), @41=((@39*@9)+(@38*@11)), @42=(@40-(@26*@41)), @43=(@41*@13), @44=(@42-(@29*@43)), @45=(@36*@14), @46=((@44*@14)-@45), @47=(@36*@1), @48=((@44*@1)-@47), @49=((@31*@14)+(@35*@1)), @50=(@37*@14), @51=((@44*@14)-@50), @52=(@37*@1), @53=((@44*@1)-@52), @54=((@19*@14)+(@32

In [34]:
model.body("link06").jacobian.local

SX(@1=sin(q_0), @2=cos(q_1), @3=(@1*@2), @4=cos(q_2), @5=sin(q_1), @6=(@1*@5), @7=sin(q_2), @8=((@3*@4)-(@6*@7)), @9=cos(q_3), @10=((@3*@7)+(@6*@4)), @11=sin(q_3), @12=((@8*@9)-(@10*@11)), @13=cos(q_4), @14=cos(q_0), @15=sin(q_4), @16=((@12*@13)+(@14*@15)), @17=-0.35, @18=(@14*@2), @19=0.218, @20=(@14*@5), @21=((@18*@4)-(@20*@7)), @22=0.057, @23=((@18*@7)+(@20*@4)), @24=0.07, @25=((@21*@9)-(@23*@11)), @26=0.0492, @27=((@25*@13)-(@1*@15)), @28=((((@17*@18)+((@19*@21)+(@22*@23)))+(@24*@25))+(@26*@27)), @29=((((@17*@3)+((@19*@8)+(@22*@10)))+(@24*@12))+(@26*@16)), @30=((@14*@13)-(@12*@15)), @31=cos(q_5), @32=((@8*@11)+(@10*@9)), @33=sin(q_5), @34=((@30*@31)+(@32*@33)), @35=((@21*@11)+(@23*@9)), @36=((@25*@15)+(@1*@13)), @37=((@35*@33)-(@36*@31)), @38=((@32*@31)-(@30*@33)), @39=((@36*@33)+(@35*@31)), @40=((@5*@4)+(@2*@7)), @41=((@2*@4)-(@5*@7)), @42=((@40*@9)+(@41*@11)), @43=(@42*@13), @44=(@42*@15), @45=((@41*@9)-(@40*@11)), @46=((@44*@31)+(@45*@33)), @47=((@45*@31)-(@44*@33)), @48=0.1035,

In [35]:
model.body("link06").jacobian.local
model.body("link06").jacobian_dt.local
model.body("link06").linear_velocity.local
model.body("link06").angular_velocity.local
model.body("link06").linear_acceleration.local
model.body("link06").angular_acceleration.local

SX(@1=sin(q_0), @2=cos(q_1), @3=(@1*@2), @4=cos(q_2), @5=sin(q_1), @6=(@1*@5), @7=sin(q_2), @8=((@3*@4)-(@6*@7)), @9=cos(q_3), @10=((@3*@7)+(@6*@4)), @11=sin(q_3), @12=((@8*@9)-(@10*@11)), @13=cos(q_4), @14=cos(q_0), @15=sin(q_4), @16=((@12*@13)+(@14*@15)), @17=(@14*@2), @18=(@14*@5), @19=((@17*@4)-(@18*@7)), @20=((@17*@7)+(@18*@4)), @21=((@19*@9)-(@20*@11)), @22=((@21*@13)-(@1*@15)), @23=((@5*@4)+(@2*@7)), @24=((@2*@4)-(@5*@7)), @25=((@23*@9)+(@24*@11)), @26=(@25*@13), @27=((@19*@11)+(@20*@9)), @28=((@8*@11)+(@10*@9)), @29=((@24*@9)-(@23*@11)), @30=(v_2+v_1), @31=(v_3+@30), @32=(@5*v_0), @33=(@2*v_0), @34=((@4*@32)+(@7*@33)), @35=((@4*@33)-(@7*@32)), @36=((@9*@34)+(@11*@35)), @37=((@15*@31)-(@13*@36)), @38=((@14*@13)-(@12*@15)), @39=((@15*@36)+(@13*@31)), @40=((@9*@35)-(@11*@34)), @41=(v_4+@40), @42=((@16*@37)+((@38*@39)+(@28*@41))), @43=(@25*@15), @44=(((@43*@39)+(@29*@41))-(@26*@37)), @45=((@42*@29)-(@44*@28)), @46=((@21*@15)+(@1*@13)), @47=((@22*@37)+((@27*@41)-(@46*@39))), @48=((@

The body jacobian and velocities can be calculated with respect to `world`, `local` and `world_aligned` frames.

In [36]:
model.body("link06").jacobian.world

SX(@1=0, @2=0.1035, @3=cos(q_0), @4=sin(q_0), @5=(-@4), @6=-0.35, @7=sin(q_1), @8=(@2-(@6*@7)), @9=cos(q_1), @10=(@3*@9), @11=(@6*@10), @12=(@4*@9), @13=(@6*@12), @14=0.057, @15=cos(q_2), @16=sin(q_2), @17=((@9*@15)-(@7*@16)), @18=0.218, @19=((@7*@15)+(@9*@16)), @20=(@8+((@14*@17)-(@18*@19))), @21=(@3*@7), @22=((@10*@15)-(@21*@16)), @23=((@10*@16)+(@21*@15)), @24=(@11+((@18*@22)+(@14*@23))), @25=(@4*@7), @26=((@12*@15)-(@25*@16)), @27=((@12*@16)+(@25*@15)), @28=(@13+((@18*@26)+(@14*@27))), @29=0.07, @30=cos(q_3), @31=sin(q_3), @32=((@26*@30)-(@27*@31)), @33=(@28+(@29*@32)), @34=((@17*@30)-(@19*@31)), @35=((@19*@30)+(@17*@31)), @36=(@20-(@29*@35)), @37=((@26*@31)+(@27*@30)), @38=((@22*@31)+(@23*@30)), @39=((@22*@30)-(@23*@31)), @40=(@24+(@29*@39)), @41=0.0492, @42=cos(q_4), @43=sin(q_4), @44=((@32*@42)+(@3*@43)), @45=(@33+(@41*@44)), @46=(@35*@42), @47=(@36-(@41*@46)), @48=((@39*@42)-(@4*@43)), @49=(@40+(@41*@48)), 
[[@1, (-(@2*@3)), (-(@8*@3)), (-(@20*@3)), ((@33*@34)-(@36*@37)), (-((@

Note that body name can be initialized with dictionary that maps given name to one presented in urdf i.e: `{'ee':'link06'}`

##### **Contacts**

In [37]:
model.body("link06").add_contact("wrench")

In [38]:
model.body("link06").contact.dim
# model.body("link06").contact.contact_frame
model.body("link06").contact.ref_frame
model.body("link06").contact.qforce

SX(@1=0.1035, @2=-0.35, @3=sin(q_1), @4=0.057, @5=cos(q_1), @6=cos(q_2), @7=sin(q_2), @8=((@5*@6)-(@3*@7)), @9=0.218, @10=((@3*@6)+(@5*@7)), @11=0.07, @12=cos(q_3), @13=sin(q_3), @14=((@10*@12)+(@8*@13)), @15=0.0492, @16=cos(q_4), @17=((((@1-(@2*@3))+((@4*@8)-(@9*@10)))-(@11*@14))-(@15*(@14*@16))), @18=cos(q_0), @19=sin(q_1), @20=cos(q_1), @21=cos(q_2), @22=sin(q_2), @23=((@20*@21)-(@19*@22)), @24=((@19*@21)+(@20*@22)), @25=((@1-(@2*@19))+((@4*@23)-(@9*@24))), @26=sin(q_0), @27=(@26*@20), @28=(@26*@19), @29=((@27*@21)-(@28*@22)), @30=((@27*@22)+(@28*@21)), @31=((@2*@27)+((@9*@29)+(@4*@30))), @32=cos(q_3), @33=sin(q_3), @34=((@29*@32)-(@30*@33)), @35=(@31+(@11*@34)), @36=((@23*@32)-(@24*@33)), @37=((@24*@32)+(@23*@33)), @38=(@25-(@11*@37)), @39=((@29*@33)+(@30*@32)), @40=sin(q_0), @41=(@40*@5), @42=(@40*@3), @43=((@41*@6)-(@42*@7)), @44=((@41*@7)+(@42*@6)), @45=((@43*@12)-(@44*@13)), @46=cos(q_0), @47=sin(q_4), @48=((((@2*@41)+((@9*@43)+(@4*@44)))+(@11*@45))+(@15*((@45*@16)+(@46*@47))))

Do not forget to rebuild the model:

In [39]:
# model.update_model()

Note how arguments are changed in dynamics related functions, i.e:

In [40]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[3],link06[3])->(dv[6]) SXFunction)

the state space representation and jacobians are changed as well:

In [41]:
model.state_space.state_derivative

SX(@1=2.47e-06, @2=0.673326, @3=-0.00025198, @4=cos(q_1), @5=2.1e-06, @6=-5.13e-06, @7=(@4*v_0), @8=sin(q_1), @9=((v_1*@7)+(@8*dv_0)), @10=0.0246611, @11=(@8*v_0), @12=((@4*dv_0)-(v_1*@11)), @13=-0.110126, @14=1.19132, @15=0.00158266, @16=(@14*((@15*@9)+(@13*@12))), @17=0.00240029, @18=9.81, @19=(@18*@8), @20=(@14*(@19+((@17*@12)-(@15*dv_1)))), @21=0.0242946, @22=-0.00062358, @23=(@14*((@13*v_1)+(@17*@11))), @24=(@14*((@17*@7)-(@15*v_1))), @25=((((@21*v_1)-(@22*@11))+(@5*@7))+((@13*@23)-(@15*@24))), @26=0.00102138, @27=(@14*((@15*@11)+(@13*@7))), @28=((((@22*v_1)-(@26*@11))+(@6*@7))-((@17*@23)+(@15*@27))), @29=cos(q_2), @30=1.332e-05, @31=(dv_2+dv_1), @32=0.00208102, @33=sin(q_2), @34=((@29*@7)-(@33*@11)), @35=((v_2*@34)+((@29*@9)+(@33*@12))), @36=0.00886621, @37=((@29*@11)+(@33*@7)), @38=(((@29*@12)-(@33*@9))-(v_2*@37)), @39=0.106092, @40=0.839409, @41=-0.35, @42=(@41*@12), @43=0.0347638, @44=(@40*(@42+((@43*@35)+(@39*@38)))), @45=-0.00541815, @46=(@41*v_1), @47=(@29*@46), @48=(@18*@4

In [42]:
model.state_space.state_jacobian

SX(@1=0.1035, @2=-0.35, @3=sin(q_1), @4=0.057, @5=cos(q_1), @6=cos(q_2), @7=sin(q_2), @8=((@5*@6)-(@3*@7)), @9=0.218, @10=((@3*@6)+(@5*@7)), @11=((@1-(@2*@3))+((@4*@8)-(@9*@10))), @12=sin(q_0), @13=sin(q_1), @14=cos(q_1), @15=cos(q_2), @16=sin(q_2), @17=((@14*@15)-(@13*@16)), @18=((@13*@15)+(@14*@16)), @19=0.07, @20=cos(q_3), @21=sin(q_3), @22=((@18*@20)+(@17*@21)), @23=0.0492, @24=cos(q_4), @25=((((@1-(@2*@13))+((@4*@17)-(@9*@18)))-(@19*@22))-(@23*(@22*@24))), @26=cos(q_3), @27=sin(q_3), @28=((@8*@26)-(@10*@27)), @29=cos(q_0), @30=(@5*@29), @31=(@3*@29), @32=((@6*@30)-(@7*@31)), @33=((@7*@30)+(@6*@31)), @34=((@2*@30)+((@9*@32)+(@4*@33))), @35=((@26*@32)-(@27*@33)), @36=(@34+(@19*@35)), @37=((@10*@26)+(@8*@27)), @38=(@11-(@19*@37)), @39=((@27*@32)+(@26*@33)), @40=cos(q_0), @41=(@14*@40), @42=(@13*@40), @43=((@15*@41)-(@16*@42)), @44=((@16*@41)+(@15*@42)), @45=((@20*@43)-(@21*@44)), @46=sin(q_4), @47=sin(q_0), @48=((((@2*@41)+((@9*@43)+(@4*@44)))+(@19*@45))+(@23*((@24*@45)-(@46*@47)))),

In [43]:
model.body("link06").contact.add_cone(mu=0.5, X=0.05, Y=0.02)

In [44]:
wrench_cone = model.body("link06").contact.cone.full()

In [47]:
model.body("link06").contact.cone.linear()

DM(
[[0, 0, -1], 
 [-1, 0, -0.5], 
 [1, 0, -0.5], 
 [0, -1, -0.5], 
 [0, 1, -0.5]])

In [52]:
model.add_body({"ee": "link06"})
model.bodies.keys()

dict_keys(['link06', 'ee'])

One can add bodies on the initialization stage based on following syntax:

In [54]:
# Symbolic(z1_description.URDF_PATH, bodies_names={'shoulder':'link03', 'ee':'link06'})

The `bodies_names` arguments can be listof body names present in urdf, however for increased readability we suggest to use the dictionary as shown above.

## **Prespecified Robots**

One can build different robots....

In [ ]:
from darli.robots import Biped, Manipulator, Quadruped

As example let us consider the Atlas humanoid robot:

Note: robot loaded in example is `fixed` in its pelvis and in real world, you have to create a floating base model to have a full set of generalized coordinates.

In [ ]:
from robot_descriptions import atlas_v4_description

biped_urdf = atlas_v4_description.URDF_PATH

biped_model = Biped(
    biped_urdf,
    torso={"torso": "pelvis"},
    foots={"left_foot": "l_foot", "right_foot": "r_foot"},
)

In [ ]:
biped_model.forward_dynamics

Function(forward_dynamics:(q[30],v[30],u[24],left_foot[6],right_foot[6])->(dv[30]) SXFunction)

In [ ]:
biped_model.inverse_dynamics

Function(inverse_dynamics:(q[30],v[30],dv[30],left_foot[6],right_foot[6])->(tau[30]) SXFunction)

In [ ]:
biped_model.body("torso").position

Function(position_torso:(q[30])->(position[3]) SXFunction)

In [ ]:
biped_model.body("torso").rotation

Function(rotation_torso:(q[30])->(rotation[3x3]) SXFunction)

In [ ]:
biped_model.body("torso").jacobian().world_aligned

Function(torso_jacobian_world_aligned:(q[30])->(jacobian[6x30]) SXFunction)

In [ ]:
biped_model.body("torso").angular_velocity().local

Function(torso_angular_velocity_local:(q[30],v[30])->(angular_velocity_local[3]) SXFunction)

In [ ]:
biped_model.body("torso").jacobian_dt().world_aligned

Function(torso_djacobian_world_aligned:(q[30],v[30])->(djacobian[6x30]) SXFunction)

In [ ]:
biped_model.body("left_foot").contact.jacobian()

{'contact_jacobian': DM(
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.0225, -0, -0.796, -0.422, 0, 0, 0, 0, 0, 0, 0, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.862, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0225, 0.05, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -0, -0, -0, 1, 0, 0, 0, 0, 0, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, -0, 0, 0, 0, 0, 0, 0]])}

In [ ]:
biped_model.body("right_foot").contact.jacobian()

{'contact_jacobian': DM(
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0225, -0, -0.796, -0.422, 0, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.862, 0, 0, 0, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.0225, 0.05, 0, 0, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -0, -0, -0, 1], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0], 
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, -0]])}

One can create a structure for specific robot, the typical reciept as follows:
* Create the child class with parent being `RobotModel`.
* Add bodies bodies and necessary contacts
* Calculate the all necessary functions with `~.update_model()` method 

An example that creates Quadruped robot possibly with arm:

In [ ]:
class Quadruped(RobotModel):
    def __init__(
        self,
        urdf_path,
        torso=None,
        foots=None,
        arm=None,
        reference="world_aligned",
        calculate=True,
    ):
        bodies_names = {}

        if torso is not None:
            if isinstance(torso, str):
                bodies_names.update([torso])
            else:
                bodies_names.update(torso)

        if arm is not None:
            if isinstance(arm, str):
                bodies_names.update([arm])
            else:
                bodies_names.update(arm)

        if foots is not None:
            bodies_names.update(foots)

        super().__init__(urdf_path, bodies_names=bodies_names, calculate=False)

        for foot in foots.keys():
            body = self.body(foot)
            body.add_contact(frame=reference, contact_type="point")

        self.set_selector(passive_joints=range(6), calculate=False)
        self.update_model()